# 基差-库存-利润分析

## 基差分析

### 基差走势

In [ ]:
import pandas as pd
df_rb_basis = pd.read_csv('data/rb-main-basis.csv')
df_rb_basis.head()

In [ ]:
import seaborn as sns
sns.lineplot(data=df_rb_basis, x='日期',y='基差')

### 基差率

### 期限结构

## 库存

### 库存周期

### 仓单

## 利润

## 综合分析